In [1]:
pip install pyepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gurobi-machinelearning

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pyepo
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import gurobi_ml


In [13]:
import time

# Iniciar temporizador para la etapa de entrenamiento
start_time_train = time.time()

In [15]:

# model for shortest path
grid = (7,7)  # grid size
optmodel = pyepo.model.grb.shortestPathModel(grid)

# generate data
num_data = 1000  # number of data
num_feat = 7  # size of feature
deg = 4  # polynomial degree
noise_width = 0.5  # noise width
x, c = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, noise_width, seed=73)

# Split the data into training and test sets
x_train, x_test, c_train, c_test = train_test_split(x, c, test_size=0.5, random_state=73)

# sklearn regressor
reg = LinearRegression()  # linear regression

# build model
twostage_model = pyepo.twostage.sklearnPred(reg)

# training
twostage_model.fit(x_train, c_train)

# Calcular el tiempo de entrenamiento
train_time = time.time() - start_time_train
print("Tiempo de entrenamiento: {:.2f} segundos".format(train_time))

# Iniciar temporizador para la etapa de predicción
start_time_predict = time.time()

# prediction
c_pred = twostage_model.predict(x_test)

# Calcular el tiempo de predicción
predict_time = time.time() - start_time_predict
print("Tiempo de predicción: {:.2f} segundos".format(predict_time))



Tiempo de entrenamiento: 0.36 segundos
Tiempo de predicción: 0.01 segundos


In [16]:

# Calculate MSE
mse = mean_squared_error(c_test, c_pred)

# Print MSE
print("MSE for Two Stage Model: ", mse)


MSE for Two Stage Model:  0.16096583759774205


In [19]:

A=optmodel.solve() # solvel

In [21]:
# Extraer la lista de soluciones del tuple
soluciones = A[0]

# Convertir la lista en un array de NumPy
soluciones_array = np.array(soluciones)

soluciones_matrix = soluciones_array.reshape(1, -1)

print("Matriz de soluciones:")
print(soluciones_matrix)

Matriz de soluciones:
[[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [23]:
print("Dimensiones de A:", soluciones_array.shape)
print("Dimensiones de c_pred:", c_pred.shape)

Dimensiones de A: (84,)
Dimensiones de c_pred: (1000, 84)


In [26]:
costs_total_pred = np.sum(soluciones_matrix * c_pred, axis=1)


In [28]:
# Calcular el costo promedio con los c_pred
average_cost_pred = np.mean(costs_total_pred)
average_cost_pred

8.784218814881381

In [30]:
#calcular costo promrdio con los costos reales
costs_total = np.sum(soluciones_matrix * c, axis=1)


In [32]:
# Calcular el costo promedio con los c_pred
average_cost = np.mean(costs_total)
average_cost

8.657135201853649

In [34]:
Regret=(average_cost-average_cost_pred)
Regret

-0.1270836130277324

In [36]:
regret_porcentual=Regret/ average_cost * 100 
regret_porcentual

-1.4679638248057107